# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import os
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

In [2]:
models = Models.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
hands = ['T8732.K7532.J.Q3', '4.AQJ98.KQ975.94', 'AQ.T6.A863.JT862', 'KJ965.4.T42.AK75']

In [4]:
from ddsolver import ddsolver
dds = ddsolver.DDSolver()

bidder_bots = [BotBid([True, False], hand, models, sampler, i, 3, dds, False) for i, hand in enumerate(hands)]

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


In [6]:
auction = ['PAD_START','PAD_START','PAD_START']  # since West deals we need 'PAD_START'

turn_i = 3 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1S', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1S', 'insta_score': 0.998}], 'hcp': [9.6, 9.6, 9.4], 'shape': [2.6, 4.0, 3.1, 3.0, 2.7, 3.9, 3.1, 3.0, 2.7, 3.9, 3.2, 3.0]}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [10.4, 9.5, 13.8], 'shape': [1.8, 2.9, 4.3, 4.0, 1.8, 3.0, 4.2, 4.0, 5.0, 2.1, 3.0, 2.9]}
{'bid': '2H', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '2H', 'insta_score': 0.999}], 'hcp': [7.7, 12.9, 7.1], 'shape': [3.1, 2.9, 2.9, 4.1, 5.2, 2.3, 2.2, 3.2, 3.2, 2.9, 2.9, 4.0]}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [12.0, 5.5, 11.4], 'shape': [5.4, 2.4, 2.7, 2.4, 3.4, 3.0, 3.4, 3.1, 2.0, 5.4, 2.8, 2.6]}
{'bid': '2S', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '2S', 'insta_score': 1.0}], 'hcp': [7.6, 12.6, 8.4], 'shape': [3.1, 3.1, 3.6, 3.2, 1.9, 5.5, 3.0, 2.6, 3.0, 3.1, 3.7, 3.1]}
{'bid': 'PASS', 'who': 'NN', 'qu